### В этом ноутбуке описан парсинг данных с сайта Auto.RU

In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import re
import time
import tqdm
from tqdm import tqdm

Парсинг данных делаем в два этапа. 

**Первым этапом** сформируем список адресов объявлений о продаже машины на Auto.RU. Для этого возмьмем первые 3000 страниц сайта объявлений. На каждой из этих страниц 38 объявлений о продаже

In [3]:
url_test = 'https://auto.ru/moskva/cars/used/?page='
url_list = []
pat = re.compile('href="(.+?)" target')
for n in range(1,3001,1):
    print(1000-n)
    url = url_test+str(n)
    response = requests.get(url)
    page = BeautifulSoup(response.text, 'html.parser')
    test_block = page.find_all('a', class_='Link ListingItemTitle-module__link')
    for element in test_block:
        res = str(element)
        res_f = pat.findall(res)
        url_list.append(res_f)

In [10]:
len(url_list)

113962

**Вторым этапом** пройдемся по адресам уже конкретных объявлений по продаже машин и возьмем оттуда всю необходимую информацию
(так как работу я выполнял на двух разных ноутбуках, мне пришлось базу с адресами, скачанными на другом ноутбуке перекинуть в этот... поэтому в коде парсера не url_list, а url_data)

In [4]:
# подготовим пустой датафрейм
# df=pd.DataFrame()

for a in tqdm(range(len(url_data))):
    url = url_data.loc[a][1][2:-2]
    while True: #используем для того, чтобы в случае обрыва связи сделать реконнект
# использую конструкцию try/except так как будут встречаться объявления с пропусками, обрыв связи и т.п.
        try:
    # делаем реквест страницы авто-ру  
            response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
            response.encoding = 'KOI-8R' # помогает избежать проблему с искажением кодировки на сайте

    # теперь будем парсить страницу
            page = BeautifulSoup(response.text, 'html.parser')

    # информация о марке машины
            model_blocks = page.find_all('div', class_='LayoutSidebar__content')
            model = model_blocks[0].contents[0].contents[0].contents[1].contents[1].text[:-1]

    #  информация о цене машины (осознание того, что объявления без цены нам не нужны пришло уже в процессе работы парсера)
            price_block = page.find_all('span', class_='OfferPriceCaption__price')
            if len(price_block)>0:
            x = price_block[0].text[:-1]
            price=int(x.replace('\xa0',''))
            else:
                price=np.NaN

    # основные характеристики машины
            main_blocks = page.find_all('ul', class_ = 'CardInfo')
            main_list = []
            for i in range(len(main_blocks[0])):
                main_list.append(main_blocks[0].contents[i].contents[0].text)
                test_text = main_blocks[0].contents[i].contents[1].text
                x2 = test_text.replace('\xa0','')
                main_list.append(x2)
            main_auto_dict = {} #в этот словарь собираем характеристики по "строкам", представленным на сайте
            # в завершенние цикла этот словарь используем для заполнения строк в датасете
            for n in range(0, len(main_list),2):
                main_auto_dict[main_list[n]] = main_list[n+1]


    # теперь комплектация машины
            complectation_blocks1 = page.find_all('div', class_='ComplectationGroups')
            complectation_list1 = []
        if len(complectation_blocks1)>0:
            for j in range(len(complectation_blocks1[0])):
                for n in range(len(complectation_blocks1[0].contents[j].contents[1].contents[0])):
                    complectation_list1.append(complectation_blocks1[0].contents[j].contents[1].contents[0].contents[n].text)
        else:
            complectation_list1 = 0
    # таким образом мы собрали всю информацию, представленную на странице. Теперь сформируем из нее строку в датасете
            data = pd.DataFrame.from_dict(main_auto_dict, orient='index')
            df1 = data.T
            df1['model'] = model
            df1['price'] = price
            df1['комплектация'] = str(complectation_list1)

    # теперь "подошьем" строку к нашему датафрейму
            df = pd.concat([df, df1], sort = False, axis = 0)
            time.sleep(0.5) #пауза, чтобы избежать слишком частых запросов на сайт и обрыва связи всвязи с этим
        except Exception as ex: # отлавливаем исключения, чтобы понять что на странице объявления не понравилось
            print('Исключение ',ex, ' на ',url,)
        except ConnectionError: # отлавливаем обрыв связи и продолжаем "стучаться" на сайт
                print('Обрыв связи на ', a)
                continue    
        break

In [29]:
len(df) # скачалась вполне приличная база

100974

In [30]:
df.to_csv('main_auto_df.csv') #сохраним скачанную базу

In [57]:
df.reset_index() # так как строки подклеивались в базу с одинаковым (нулевым) индексом

,index,год выпуска,Пробег,Кузов,Цвет,Двигатель,Налог,Коробка,Привод,Руль,...,Владение,Таможня,Гарантия,VIN,Госномер,марка,цена,комплектация,Обмен,Кузов №
0,0,2016,62000км,хэтчбек 5 дв.,синий,1.6 л / 130л.с. / Бензин,4550₽ / год,механическая,передний,Левый,...,3 года и 2 месяца,Растаможен,До мая 2021,XWE**************,******|777,Kia,820000,"['AUX', 'USB', 'Аудиосистема', 'Bluetooth', 'Р...",NaN,NaN
1,0,2013,135835км,седан,чёрный,1.4 л / 122л.с. / Бензин,4148₽ / год,роботизированная,передний,Левый,...,3 года и 2 месяца,Растаможен,NaN,WAU**************,******|799,Audi,795000,0,Рассмотрю варианты,NaN
2,0,2003,189000км,внедорожник 5 дв.,синий,2.5 л / 210л.с. / Бензин,13650₽ / год,автоматическая,полный,Левый,...,NaN,Растаможен,NaN,YV1**************,******|763,Volvo,499000,0,NaN,NaN
3,0,2017,23150км,внедорожник 5 дв.,белый,2.0 л / 211л.с. / Бензин,13715₽ / год,роботизированная,полный,Левый,...,3 года и 3 месяца,Растаможен,NaN,SJK**************,******|799,Infiniti,1945000,"['Круиз-контроль', 'Мультифункциональное рулев...",NaN,NaN
4,0,2019,8512км,седан,чёрный,2.5 л / 180л.с. / Бензин,9000₽ / год,автоматическая,передний,Левый,...,NaN,Растаможен,NaN,XWE**************,NaN,Hyundai,1849000,"['Круиз-контроль', 'Мультифункциональное рулев...",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,2015,77000км,седан,чёрный,1.8 л / 190л.с. / Бензин,9500₽ / год,роботизированная,передний,Левый,...,NaN,Растаможен,NaN,WAU**************,NaN,Audi,1225000,"['Круиз-контроль', 'Мультифункциональное рулев...",Рассмотрю варианты,NaN
96,0,2016,30902км,внедорожник 5 дв.,серый,3.7 л / 314л.с. / Бензин,47100₽ / год,автоматическая,полный,Левый,...,NaN,Растаможен,NaN,Y3W**************,NaN,Cadillac,2545000,"['Круиз-контроль', 'Электростеклоподъёмники пе...",Рассмотрю варианты,NaN
97,0,2013,15958км,купе,серый,6.3 л / 740л.с. / Бензин,111000₽ / год,роботизированная,задний,Левый,...,NaN,Растаможен,NaN,ZFF**************,NaN,Ferrari,15000000,"['Круиз-контроль', 'Электростеклоподъёмники пе...",NaN,NaN
98,0,2014,92000км,внедорожник 5 дв.,коричневый,3.5 л / 294л.с. / Бензин,44100₽ / год,автоматическая,полный,Левый,...,NaN,Растаможен,NaN,Z6F**************,NaN,Ford,1435000,"['Круиз-контроль', 'Электрорегулировка руля', ...",Рассмотрю варианты,NaN


In [61]:
df.columns

Index(['год выпуска', 'Пробег', 'Кузов', 'Цвет', 'Двигатель', 'Налог',
       'Коробка', 'Привод', 'Руль', 'Состояние', 'Владельцы', 'ПТС',
       'Владение', 'Таможня', 'Гарантия', 'VIN', 'Госномер', 'марка', 'цена',
       'комплектация', 'Обмен', 'Кузов №'],
      dtype='object')

А теперь сделаем **третий этап**. 

Перескачаем тестовую базу Kaggle в том типе, который получился у нас при скачивании тренинговой базы. Список адресов объявлений у нас есть, так что пройдемся по ним тем же парсером, которым скачивали тренинговую базу. Таким образом у нас должны получиться однотипные базы и не будет расхождения между количеством представленных данных.

(при скачивании обнаружилось несколько удаленных с сайта объявлений, так что информацию по этим объявлениям пришлось перенести из базы Kaggle)

In [32]:
data_kaggle = pd.read_csv('data_kaggle.csv')

In [33]:
len(data_kaggle.car_url)

34686

парсим тестовую базу каггл

In [50]:
df_kaggle=pd.DataFrame()

# забираем адрес страницы из тестовой базы каггла
for a in tqdm(range(len(data_kaggle.car_url))):
    while True: #используем для того, чтобы при разрыве связи продолжить скачивание с того же места!
        try:
            url = data_kaggle.car_url.iloc[a]

        # делаем реквест страницы авто-ру  
            response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
            response.encoding = 'KOI-8R'

        # теперь будем парсить страницу
            page = BeautifulSoup(response.text, 'html.parser')

        # марка машины
        #     model_blocks = page.find_all('div', class_='LayoutSidebar__content')
        #     model = model_blocks[0].contents[0].contents[0].contents[1].contents[1].text[:-1]
            model = data_kaggle.brand.iloc[a]
        # цена машины нам не нужна - там все машины проданы...  
        #     price_block = page.find_all('span', class_='OfferPriceCaption__price')
        #     if len(price_block)>0:
        #         x = price_block[0].text[:-1]
        #         price=int(x.replace('\xa0',''))
        #     else:
        #         price=np.NaN

        # основные характеристики машины
            main_blocks = page.find_all('ul', class_ = 'CardInfo')
            main_list = []
            if len(main_blocks)>0:
                for i in range(len(main_blocks[0])):
                    main_list.append(main_blocks[0].contents[i].contents[0].text)
                    test_text = main_blocks[0].contents[i].contents[1].text
                    x2 = test_text.replace('\xa0','')
                    main_list.append(x2)
                main_auto_dict = {}
                for n in range(0, len(main_list),2):
                    main_auto_dict[main_list[n]] = main_list[n+1]
            else: #на тот случай, если страница удалена или информация не представлена
                main_auto_dict = {'год выпуска': data_kaggle.modelDate.iloc[a],'Пробег': data_kaggle.mileage.iloc[a],
                                  'Кузов': data_kaggle.bodyType.iloc[a],
                                  'Цвет': data_kaggle.color.iloc[a],
                                  'Двигатель': [data_kaggle.engineDisplacement.iloc[a], data_kaggle.enginePower.iloc[a],data_kaggle.fuelType.iloc[a]],
                                  'Налог': np.nan,
                                  'Коробка': data_kaggle.vehicleTransmission.iloc[a],
                                  'Привод': data_kaggle['Привод'].iloc[a],
                                  'Руль': data_kaggle['Руль'].iloc[a],
                                  'Состояние': data_kaggle['Состояние'],
                                  'Владельцы': data_kaggle['Владельцы'].iloc[a],
                                  'ПТС': data_kaggle['ПТС'].iloc[a],
                                  'Таможня': data_kaggle['Таможня'].iloc[a],
                                  'Обмен': np.nan,
                                  'VIN': np.nan}


        # теперь комплектация машины
            complectation_blocks1 = page.find_all('div', class_='ComplectationGroups')
            complectation_list1 = []
            if len(complectation_blocks1)>0:
                for j in range(len(complectation_blocks1[0])):
                    for n in range(len(complectation_blocks1[0].contents[j].contents[1].contents[0])):
                        complectation_list1.append(complectation_blocks1[0].contents[j].contents[1].contents[0].contents[n].text)
            else: #на тот случай, если страница удалена или информация не представлена
                complectation_list1 = a
            data = pd.DataFrame.from_dict(main_auto_dict, orient='index')
            df1 = data.T
            df1['марка'] = model
        #     df1['price'] = price
            df1['комплектация'] = str(complectation_list1)

        # теперь "подошьем" табличку к нашему датафрейму
            df_kaggle = pd.concat([df_kaggle, df1], sort = False, axis = 0)
        except ConnectionError:
            print('Обрыв связи на ', a)
            continue
        break

100%|██████████| 34686/34686 [9:14:16<00:00,  1.04it/s]   


In [52]:
df_kaggle.to_csv('df_kaggle.csv') # сохраним полученную базу

Таким образом мы скачали две базы данных. Тренинговую и тестовую. Они скачаны одним парсером, то есть данные в них однотипны, расположены одинаково и т.п. Обработкой этих баз займемся в других ноутбуках